1. Import Libaries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
# from google.colab import drive
# drive.mount("/content/gdrive",force_remount=True)
matches = pd.read_csv('./matches.csv')
matches.info()
matches.shape
matches.head()

2. Assess Missing Values

In [ ]:
# Checking the missing values in the dataset
matches.isnull().sum()
matches[pd.isnull(matches['winner'])]
matches['winner'].fillna('Draw', inplace=True)
#Find cities which are null
matches[pd.isnull(matches['city'])]
matches['city'].fillna('Dubai',inplace=True)
matches.isnull().sum()
"""# Rename teams"""
matches.replace(['Mumbai Indians','Kolkata Knight Riders','Royal Challengers Bangalore','Deccan Chargers','Chennai Super Kings','Rajasthan Royals','Delhi Daredevils',"Delhi Capitals",'Gujarat Lions','Kings XI Punjab','Sunrisers Hyderabad','Rising Pune Supergiants',"Rising Pune Supergiant",'Kochi Tuskers Kerala','Pune Warriors'],['MI','KKR','RCB','DC','CSK','RR','DD','DD','GL','KXIP','SRH','RPS','RPS','KTK','PW'],inplace=True)
matches.head()

3. Exploratory Data Analysis

In [ ]:
# Venues
"""
print(matches['city'].unique())
len(matches['city'].unique())
"""# Teams"""
teams = matches['team1'].unique()
teams
"""# Most Player Of the Match"""
matches['player_of_match'].value_counts()
"""## Toss Decision by teams season wise"""
sns.countplot(x='season',hue='toss_decision',data=matches)
"""# Most toss winner"""
matches['toss_winner'].value_counts().plot(kind='bar')
"""# Most Wins by team"""
wins=pd.DataFrame(matches['winner'].value_counts()).reset_index()
wins.columns=['team_name','wins']
wins
"""## Teams Win Percentage"""
matches_played_byteams=pd.concat([matches['team1'],matches['team2']],a
xis=1)
teams=(matches_played_byteams['team1'].value_counts()+matches_played
_byteams['team2'].value_counts()).reset_index()
teams.columns=['team_name','Matches_played']
teams
player=teams.merge(wins,left_on='team_name',right_on='team_name',how
='inner')
player.columns=['team','matches_played','wins']
player
player['%win']=(player['wins']/player['matches_played'])*100
player
"""# Favourable Ground For Each Team"""
def favorable(df,team_name):return df[df['winner']==team_name]['venue'].value_counts().nlargest(5)
favorable(matches,'RCB').plot(kind='bar')

4. Encoding Categorical Values

In [ ]:
encode = {'team1':{'MI':1,'KKR':2,'RCB':3,'DC':4,'CSK':5,'RR':6,'DD':7,'GL':8,'KXIP':9,'SRH':10,'RPS':11,'KTK':12,'PW':13},'team2':{'MI':1,'KKR':2,'RCB':3,'DC':4,'CSK':5,'RR':6,'DD':7,'GL':8,'KXIP':9,'SRH':10,'RPS':11,'KTK':12,'PW':13},'toss_winner':{'MI':1,'KKR':2,'RCB':3,'DC':4,'CSK':5,'RR':6,'DD':7,'GL':8,'KXIP':9,'SRH':10,'RPS':11,'KTK':12,'PW':13},'winner':{'MI':1,'KKR':2,'RCB':3,'DC':4,'CSK':5,'RR':6,'DD':7,'GL':8,'KXIP':9,'SRH':10,'RPS':11,'KTK':12,'PW':13,'Draw':14}}
matches.replace(encode, inplace=True)
matches.head()
"""# Encoding city"""
cat_list=matches["city"]
encoded_data, mapping_index = pd.Series(cat_list).factorize()
print("Visakhapatnam :",mapping_index.get_loc("Visakhapatnam"))
mapping_index
"""# Encoding Venue"""
cat_list1=matches["venue"]
encoded_data1, mapping_index1 = pd.Series(cat_list1).factorize()
# print(mapping_index1)
print("Dr. Y.S. Rajasekhara Reddy ACA-VDCA CricketStadium",mapping_index1.get_loc("Dr. Y.S. Rajasekhara ReddyACA-VDCA Cricket Stadium"))
mapping_index1
"""# Encoding Toss Decision"""
cat_list2=matches["toss_decision"]
encoded_data2, mapping_index2 = pd.Series(cat_list2).factorize()
#print(encoded_data2)
print(mapping_index2)
print("Field : ",mapping_index2.get_loc("field"))
print("Bat : ",mapping_index2.get_loc("bat"))

5. Feature Selection

In [ ]:
matches = matches[matches['season']<=2019]
matches
"""**Dropping features which are not useful for prediction**"""
matches =matches[['team1','team2','city','toss_decision','toss_winner','venue','winner']
]
df = pd.DataFrame(matches)
df = df.reset_index(drop = True)
df
"""**Building predictive model , convert categorical to numerical
data**"""
for i in range(len(df)):
    if df['winner'][i]==df['team1'][i]:df['winner'][i]=0
    else:df['winner'][i]=1
from sklearn.preprocessing import LabelEncoder
var_mod = ['city','toss_decision','venue']
le = LabelEncoder()
for i in var_mod:df[i] = le.fit_transform(df[i])
df
df.corr()
from sklearn.preprocessing import StandardScaler
scaled_features = df.copy()
col_names = ['team1', 'team2', 'venue', 'toss_winner','city','toss_decision']
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
df[col_names] = features
print(df)

6. Model Building

In [ ]:
#Import models from scikit learn module:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
def recall(tp,fn):return (tp/(tp+fn))
def precision(tp,fp):return (tp/(tp+fp))
def fscore(tp,fn,fp):recall_value = recall(tp,fn)
precision_value = precision(tp,fp)
return ((2 * recall_value * precision_value ) / ( recall_value +precision_value ))
#Generic function for making a classification model and accessing
performance:def classification_model(model, data, predictors, outcome):trained_model = model.fit(data[predictors],data[outcome])
return trained_model
def accuracy_of_model(trained_model, data, predictors, outcome):predictions = trained_model.predict(data[predictors])
accuracy = metrics.accuracy_score(predictions,data[outcome])
print('Accuracy : %s' % '{0:.3%}'.format(accuracy))
tn, fp, fn, tp = confusion_matrix(data[outcome], predictions).ravel()
# print("True Positive :",tp)
# print("True Negative :",tn)
# print("False Positive :",fp)
# print("True Neagtive :",tp)
# print("Recall :",recall(tp,fn))
# print("Precision :",precision(tp,fp))
# print("F-score :",fscore(tp,fn,fp))
# plot_confusion_matrix(trained_model, data[predictors],data[outcome])
# plt.show()
# print(predictions)
return predictions
"""**Naive Bayes**"""
from sklearn.naive_bayes import GaussianNB
outcome_var="winner"
predictor_var = ['team1', 'team2', 'venue',
'toss_winner','city','toss_decision']
model = GaussianNB()
NB_model = classification_model(model, df,predictor_var,outcome_var)
"""**Support Vector Machine**"""
#SVM
from sklearn import svm
model = svm.SVC(kernel='linear',gamma = 2)
outcome_var="winner"
predictor_var = ['team1', 'team2', 'venue','toss_winner','city','toss_decision']
SVM_linear_model = classification_model(model,df,predictor_var,outcome_var)
model = svm.SVC(kernel='rbf',gamma = 2)
SVM_rbf_model = classification_model(model,
df,predictor_var,outcome_var)
with open('matches.pkl','wb') as f:pickle.dump(SVM_rbf_model,f)
""" **KNN**"""
#applying knn algorithm
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
outcome_var = "winner"
predictor_var = ['team1', 'team2', 'venue','toss_winner','city','toss_decision']
knn_3_model = classification_model(model,df,predictor_var,outcome_var)

7. Model Testing

In [ ]:
import pandas as pd
test_data_array = [0]*13
test_data = pd.DataFrame()
for year in range(2008,2021):
    if year==2020:
        test = pd.read_csv('./IPL Matches 2008-2020.csv')
    else:
        test = pd.read_csv('./matches.csv')
for i in range(len(test['date'])):
    test['date'][i]=test['date'][i][:4]
    test_length = len(test)
for i in range(test_length ):
    if test['winner'][i]==test['team1'][i]:
        test['winner'][i]=0
    else:
        test['winner'][i]=1
if year == 2020:
    test = test.loc[test['date']==str(year)]
else:
    test = test[test['season']==year]
test = test[['team1', 'team2', 'venue','toss_winner','city','toss_decision','winner']]
test.replace(['Mumbai Indians','Kolkata Knight Riders','Royal ChallengersBangalore','Deccan Chargers','Chennai Super Kings','Rajasthan Royals','Delhi Daredevils',"Delhi Capitals",'GujaratLions','Kings XI Punjab','Sunrisers Hyderabad','Rising Pune Supergiants',"Rising PuneSupergiant",'Kochi Tuskers Kerala','Pune Warriors'],['MI','KKR','RCB','DC','CSK','RR','DD','DD','GL','KXIP','SRH','RPS','RPS','KTK','PW'],inplace=True)
encode = {'team1':{'MI':1,'KKR':2,'RCB':3,'DC':4,'CSK':5,'RR':6,'DD':7,'GL':8,'KXIP':9,'SRH':10,'RPS':11,'KTK':12,'PW':13},'team2':{'MI':1,'KKR':2,'RCB':3,'DC':4,'CSK':5,'RR':6,'DD':7,'GL':8,'KXIP':9,'SRH':10,'RPS':11,'KTK':12,'PW':13},'toss_winner':{'MI':1,'KKR':2,'RCB':3,'DC':4,'CSK':5,'RR':6,'DD':7,'GL':8,'KXIP':9,'SRH':10,'RPS':11,'KTK':12,'PW':13}}
test.replace(encode, inplace=True)
from sklearn.preprocessing import LabelEncoder
var_mod = ['city','toss_decision','venue']
le = LabelEncoder()
test = test.fillna(value = {'city':"Dubai"})
for i in var_mod:
    test[i] = le.fit_transform(test[i])
test_data_array[year-2008]=test
test_data = pd.concat(test_data_array,ignore_index=True)
print((test_data))
from sklearn.preprocessing import StandardScaler
for i in range(13):
    scaled_features = test_data_array[i].copy()
col_names = ['team1', 'team2', 'venue', 'toss_winner','city','toss_decision']
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
test_data_array[i][col_names] = features
test_data_array[11]
all_predictions = []
#SVM Linear
outcome_var=["winner"]
predictor_var = ['team1', 'team2', 'venue',
'toss_winner','city','toss_decision']
for i in range(2019,2021):
    print("Year :", i)
accuracy_of_model(SVM_linear_model , test_data_array[i-2008] ,predictor_var, outcome_var)
#SVM RBF
outcome_var=["winner"]
predictor_var = ['team1', 'team2', 'venue',
'toss_winner','city','toss_decision']
for i in range(2019,2021):
    print("Year :", i)
accuracy_of_model(SVM_rbf_model , test_data_array[i-2008] ,predictor_var, outcome_var)
#Naive Baye's
outcome_var=["winner"]
predictor_var = ['team1', 'team2', 'venue',
'toss_winner','city','toss_decision']
for i in range(2019,2021):
    print("Year :", i)
accuracy_of_model(NB_model , test_data_array[i-2008] , predictor_var,outcome_var)
#KNN
outcome_var=["winner"]
predictor_var = ['team1', 'team2', 'venue',
'toss_winner','city','toss_decision']
for i in range(2019,2021):
    print("Year :", i)
accuracy_of_model(knn_3_model , test_data_array[i-2008] ,predictor_var, outcome_var)